<a href="https://colab.research.google.com/github/gagc056/neural_networks/blob/main/redes_neuronales_reg_clas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import pandas as pd

concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

concrete_data.shape

concrete_data.describe()
concrete_data.isnull().sum()

#Split data into predictors and target
#The target variable in this problem is the concrete sample strength. 
#Therefore, our predictors will be all the other columns.
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

#Let's do a quick sanity check of the predictors and the target dataframes.
predictors.head()
target.head()

#Finally, the last step is to normalize the data by substracting the mean and dividing
#by the standard deviation.
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

#Let's save the number of predictors to _n_cols_ since we will need this number when building our network.
n_cols = predictors_norm.shape[1] # number of predictors

## Import Keras
import keras
from keras.models import Sequential
from keras.layers import Dense

#build a neural network
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Train and Test the Network
# build the model
model = regression_model()

# fit the model
model.fit(predictors_norm, target, validation_split=0.2, epochs=100, verbose=2)




In [ ]:
import keras

from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

# import the data
from keras.datasets import mnist

# read the data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train.shape

plt.imshow(X_train[0])

#With conventional neural networks, we cannot feed in the image as input as is.
# So we need to flatten the images into one-dimensional vectors, each of size 1 x (28 x 28) = 1 x 784.
num_pixels = X_train.shape[1] * X_train.shape[2] # find size of one-dimensional vector

X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32') # flatten training images
X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32') # flatten test images

#Since pixel values can range from 0 to 255, let's normalize the vectors to be between 0 and 1.
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

#Finally, before we start building our model, remember that for classification we need to divide our target variable into categories. 
#We use the to_categorical function from the Keras Utilities package.
# one hot encode outputs
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

num_classes = y_test.shape[1]
print(num_classes)

# Build a Neural Network
# define classification model
def classification_model():
    # create model
    model = Sequential()
    model.add(Dense(num_pixels, activation='relu', input_shape=(num_pixels,)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    
    # compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

## Train and Test the Network
# build the model
model = classification_model()

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, verbose=2)

# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)

#Let's print the accuracy and the corresponding error.
print('Accuracy: {}% \n Error: {}'.format(scores[1], 1 - scores[1]))   

#Just running 10 epochs could actually take over 20 minutes. But enjoy the results as they are getting generated.

#Sometimes, you cannot afford to retrain your model everytime you want to use it, especially if you are limited on 
#computational resources and training your model can take a long time. Therefore, with the Keras library, you can save your model after training. 
#To do that, we use the save method.
model.save('classification_model.h5')


#Since our model contains multidimensional arrays of data, then models are usually saved as .h5 files.

#When you are ready to use your model again, you use the load_model function from keras.models.
from keras.models import load_model
pretrained_model = load_model("classification_model.h5")
​




